In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import torch.optim as optim
import yaml
import pickle
from torch.utils.data import DataLoader
from data import dataset
import pdb

Carico il configuratore che tiene traccia di tutte le informazioni generali 

In [2]:
with open("/home/andrea/Scrivania/Tesi/leonardo/config_env.yaml", 'r') as f:
    config_env = yaml.safe_load(f)

id_model = "GCN-LSTM"
with open(os.path.join(config_env['paths']['config'], f"{id_model}.yaml"), 'r') as f:
    config = yaml.safe_load(f)
    
config.update(config_env)

if 'epochs' in config_env['setting'].keys():
    config['training']['epochs'] = config_env['setting']['epochs']


In [3]:
past_step = config['setting']['past_step']
future_step = config['setting']['future_step']
batch_size= 4#config['setting']['batch_size']

with open( os.path.join(config['paths']['data'],config['setting']['dataset'], f"{past_step}_{future_step}.pkl"), 'rb') as f:
    dataset = pickle.load(f)
len_train = int(len(dataset)*0.75)
len_val = len(dataset)-len_train
df_train, df_val = torch.utils.data.random_split(dataset=dataset, lengths = [len_train, len_val])
dl_train = DataLoader(dataset=df_train, batch_size=batch_size, shuffle=True)
dl_val = DataLoader(dataset=df_val, batch_size=batch_size, shuffle=True)
batch, y, adj = next(iter(dl_train))
config['setting']['in_feat'] = batch.shape[-1]
print(batch.shape)
print(y.shape)


torch.Size([4, 30, 107, 23])
torch.Size([4, 50, 107])


In [5]:
class embedding_layer(torch.nn.Module):
    def __init__(self,
                 categorical:list,
                 dim_categorical:int, 
                 concat:bool):
        
        super(embedding_layer, self).__init__()
        self.concat = concat
        self.embedding = nn.ModuleList([nn.Embedding(categorical[i], dim_categorical) for i in range(len(categorical))])
    def forward(self, x):
    
        out = 0.0
        # Con l'unsqueeze creo una nuova dimensione ed è come se mi salvassi per ogni features di ogni nodo di ogni batch tutti gli embedding
        # Questo solo se devo sommare tutti gli embedding

        for i in range(len(self.embedding)):
            out += self.embedding[i](x[:,:,:, i])    
        return out.float()
        
class pre_processing(torch.nn.Module):
    def __init__(self,
                 in_feat:int, 
                 out_feat:int, 
                 dropout:float):
        
        super(pre_processing, self).__init__()
        self.linear = nn.Sequential(nn.Dropout(dropout),
                                    nn.Linear(in_features = in_feat, out_features = 128),
                                    nn.ReLU(), 
                                    nn.Linear(in_features = 128, out_features = 128),
                                    nn.ReLU(),
                                    nn.Linear(in_features = 128, out_features = out_feat, bias=False))
        
        
    def forward(self, x):  
        return self.linear(x.float())

class my_gat(torch.nn.Module):

    def __init__(self, 
                 in_channels: int, 
                 out_channels: int,
                 past:int, 
                 dim_hidden_emb:int = 128):
        super(my_gat, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.past = past 
        self.lin = nn.Linear(in_channels, 
                             out_channels, 
                             bias = False)
        self.a = nn.Parameter(torch.randn(in_channels*2))
        self.emb = nn.Linear(in_features = in_channels, 
                             out_features = dim_hidden_emb, 
                             bias = False)
        

    def forward(self,
                x0: tuple) -> torch.tensor:
        
        x, A = x0   
        x_emb = self.emb(x)
        B, S, N, _ = x.shape
        ## (h_i||h_j)
        z = torch.stack([torch.concat((x_emb[:, :,i],x_emb[:,:, j]), -1) for j in range(N) for i in range(N)], -2)
        z = F.leaky_relu(torch.matmul(z, self.a).view(B,S, N, N)) # B, S, N**2
        #pi = torch.einsum('bdjk,bdik->bdij', x_emb, x_emb)

        # Apply the mask to fill values in the input tensor
        # sigmoid(Pi*X*W)
        pi = F.softmax(z.masked_fill(A == 0., -float('infinity')), -1)
        x = torch.einsum('bpik,bpkj->bpij', pi, x)
        x = F.sigmoid(self.lin(x))
        
        return (x, A)

class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        # LSTM weights and biases
        self.W_i = nn.Linear(input_size + hidden_size, hidden_size)
        self.W_f = nn.Linear(input_size + hidden_size, hidden_size)
        self.W_o = nn.Linear(input_size + hidden_size, hidden_size)
        self.W_g = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, x, hidden):
        h_prev, c_prev = hidden
        
        # Concatenate input and previous hidden state
        combined = torch.cat((x, h_prev), dim=-1)

        # Compute the input, forget, output, and cell gates
        i = torch.sigmoid(self.W_i(combined))
        f = torch.sigmoid(self.W_f(combined))
        o = torch.sigmoid(self.W_o(combined))
        g = torch.tanh(self.W_g(combined))

        # Update the cell state and hidden state
        c = f * c_prev + i * g
        h = o * torch.tanh(c)

        return h, c
        
        
class GAT_LSTMseq2seq(torch.nn.Module):
    def __init__(self, 
                 in_feat:int, 
                 past: int, 
                 future: int,
                 categorical:list,
                 device, 
                 out_preprocess:int = 128, 
                 dropout: float = 0.1, 
                 dim_categorical:int = 64, 
                 concat:bool = True,
                 num_layer_gnn:int = 1,
                 hidden_gnn:int = 128,
                 hidden_lstm: int = 128, 
                 hidden_propagation:int = 128):
        
        super(GAT_LSTMseq2seq, self).__init__()
        
        self.in_feat = in_feat         # numero di features di ogni nodo prima del primo GAT        
        self.past = past 
        self.future = future
        self.hidden_gnn = hidden_gnn
        self.hidden_lstm = hidden_lstm
        self.device = device
        self.categorical = categorical
        ########## PREPROCESSING PART #############        
        # preprossessing dell'input
    
        self.embedding = embedding_layer(categorical = categorical,
                                           dim_categorical = dim_categorical, 
                                           concat = concat)
        in_feat_preprocessing = in_feat + dim_categorical - len(categorical)
        self.pre_processing = pre_processing(in_feat = in_feat_preprocessing, 
                                             out_feat = out_preprocess, 
                                             dropout = dropout)
        
        ########## FIRST GNN PART #############
        # LA CGNN mi permette di vedere spazialmente la situazione circostante
        # più layer di CGNN più spazialmente distante arriva l'informazione
        # B x P x N x F
    
        layers = []
        
        for i in range(num_layer_gnn):
            in_channels = out_preprocess if i == 0 else hidden_gnn
            layers.append(my_gat(in_channels = in_channels, 
                                  out_channels = hidden_gnn, 
                                  past = past))            
        self.gnn = nn.Sequential(*layers)

        self.lstm = LSTMCell(input_size = hidden_gnn, 
                             hidden_size = hidden_lstm)
        
        self.lstm_future = LSTMCell(input_size = 1 + dim_categorical, 
                                    hidden_size = hidden_lstm)
        print(self.lstm_future)
        
        self.decoding = nn.Sequential(nn.Linear(in_features = hidden_lstm, 
                                                out_features = hidden_propagation), 
                                      nn.ReLU(), 
                                      nn.Linear(in_features = hidden_propagation,
                                                out_features = hidden_propagation),
                                      nn.ReLU(),
                                      nn.Linear(in_features = hidden_propagation,
                                                out_features = 1))
               
    def forward(self, x, adj):
        ########### pre-processing dei dati ##########
        emb = self.embedding(x[:,:,:,-len(self.categorical):].int())
        x = torch.cat((x[:,:,:,:-len(self.categorical)], emb), -1)
        x = self.pre_processing(x)
        
        ########## GNN processing ######################
        x, _ = self.gnn((x, adj))

        ########## LSTM part ###########################
        batch_size, seq_len, nodes, features = x.size()
        h, c = [torch.zeros(batch_size, nodes, self.hidden_lstm).to(x.device)] * 2
        
        for t in range(seq_len):
            h, c = self.lstm(x[:, t], (h, c))  
        out = [self.decoding(h).unsqueeze(1)]
        h, c = [torch.zeros(batch_size, nodes, self.hidden_lstm).to(x.device)] * 2
        pdb.set_trace()
        for i in range(self.future-1):
            x_t = torch.cat((out[-1], emb), -1)
            h, c = self.lstm_future(x_t[:,0], (h,c))
            out.append(self.decoding(h).unsqueeze(1))
        out = torch.cat(out, 1).flatten(-2)
        return  out
        
model = GAT_LSTMseq2seq(in_feat = batch.shape[-1], 
              past = past_step,
              future = future_step,
              categorical = config['model']['categorical'],
              device = torch.device('cpu'))
yh = model(batch.to(model.device), adj[0].to(model.device))
yh.shape

LSTMCell(
  (W_i): Linear(in_features=320, out_features=128, bias=True)
  (W_f): Linear(in_features=320, out_features=128, bias=True)
  (W_o): Linear(in_features=320, out_features=128, bias=True)
  (W_g): Linear(in_features=320, out_features=128, bias=True)
)
> /tmp/ipykernel_107369/1981110094.py(194)forward()
    192         h, c = [torch.zeros(batch_size, nodes, self.hidden_lstm).to(x.device)] * 2
    193         pdb.set_trace()
--> 194         for i in range(self.future-1):
    195             x_t = torch.cat((out[-1], emb[:,:1,:,self.in_feat-len(self.categorical):]), -1)
    196             h, c = self.lstm_future(x_t[:,0], (h,c))



ipdb>  out[-1].shape


torch.Size([4, 1, 107, 1])


ipdb>  emb[:,:1,:,self.in_feat-len(self.categorical):].shape


torch.Size([4, 1, 107, 43])


ipdb>  emb.shape


torch.Size([4, 30, 107, 64])


ipdb>  q


In [6]:
batch.shape

torch.Size([4, 30, 107, 23])

In [ ]:
def step(model, 
         dataloader,
         optimizer,
         criterion,
         training: bool = False):

    if training:
        model.train()
    else:
        model.eval()

    is_train = True if training else False
    loss_epoch = 0.0
    with torch.set_grad_enabled(is_train):
        for x, y, adj in iter(dataloader):
            x = x.to(model.device).float()
            y = y.to(model.device).float()
            yh = model(x, adj.to(model.device))
            loss = criterion(yh, y)
    
            # Backward and optimize
            if training:
                loss.backward()
                optimizer.step()
                
                # blocking the gradient summation 
                optimizer.zero_grad()
    
    
            loss_epoch += loss.item()
    loss_epoch/=len(dataloader)

    return loss_epoch


def training(model, 
             train_loader, 
             val_loader, 
             num_epochs, 
             criterion,
             optimizer):
    loss_train = []
    loss_val = []
    be = np.inf
    bm = model
    
    for epoch in tqdm(range(num_epochs)):
            
        # Calculate average loss for the epoch
        loss = step(model = model, 
                         dataloader = train_loader, 
                         optimizer = optimizer, 
                         criterion = criterion,
                         training = True)
        
        loss_train.append(loss)
        
        loss = step(model = model, 
                         dataloader = val_loader, 
                         optimizer = optimizer, 
                         criterion = criterion,
                         training = False)
        
        loss_val.append(loss)
        
        if (loss_val[-1]<be) & (epoch/num_epochs>0.15):
            be = loss_val[-1]
            bm = model
        if (epoch+1)%10 == 0:
            print(f"loss train epoch {epoch+1} == {loss_train[-1]}")
            print(f"loss val epoch {epoch+1} == {loss_val[-1]}")
    return bm, loss_train, loss_val

def training_not_aggregate(model, 
                           dl_train:dict, 
                           dl_val:dict, 
                           num_epochs:int, 
                           criterion,
                           device,
                           optimizer):
    loss_train = []
    loss_val = []
    be = np.inf
    bm = model
    
    for epoch in tqdm(range(num_epochs)):
        loss_train.append(0)
        loss_val.append(0)
        
        for key in dl_val.keys():   
            # Calculate average loss for the epoch
            loss = step(model = model, 
                        dataloader = dl_train[key], 
                        optimizer = optimizer, 
                        device = device,
                        criterion = criterion,
                        training = True)
            
            loss_train[-1]+= loss
            
            loss = step(model = model, 
                             dataloader = dl_val[key], 
                             optimizer = optimizer, 
                             device = device,
                             criterion = criterion,
                             training = False)
            
            loss_val[-1]+= loss
            
        loss_train[-1] = loss_train[-1]/len(dl_val.keys())
        loss_val[-1] = loss_val[-1]/len(dl_val.keys())
        if (loss_val[-1]<be) & (epoch/num_epochs>0.15):
            be = loss_val[-1]
            bm = model
        if (epoch+1)%10 == 0:
            print(f"loss train epoch {epoch+1} == {loss_train[-1]}")
            print(f"loss val epoch {epoch+1} == {loss_val[-1]}")
    return bm, loss_train, loss_val


## Loss of the model
As always, the model must perform well on average on all points. 
$$
\frac{1}{n|T|}\sum_{i=1}^n\sum_{j}^{|T|}\left|f\left(t_j,x_{i,t_j}\right)-\hat{f}\left(t_j,x_{i,t_j}\right)\right|
$$
In addition, it is important that the model did not have, at worst, too high an error.
$$
\left(\int_X\left(\int_T|f(t,x)-\hat{f}(t,x)|\,dt\right)^p\,dx\right)^{\frac{1}{p}}\to\max_{x\in X}\int_T|f(t,x)-\hat{f}(t,x)|\,dt
$$

In [6]:
#device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GLSTM(in_feat = batch.float().shape[-1], 
              past = past_step,
              future = future_step,
              categorical = config['model']['categorical'],
              device = device).to(device)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def linfty(y, yh, alpha = 7e-5):
    out = F.l1_loss(y,yh, reduction = "mean")
    #out += alpha*torch.max(torch.sum(torch.abs(y-yh), (-2,-1)))
    out += alpha*torch.max(torch.abs(y-yh))
    return out
optimizer = optim.Adam(model.parameters(), 
                       lr = 1e-4, 
                       weight_decay = 2e-4)
model.to(device)
model.device = device
model, loss_training, loss_validation = training(model, 
                                             train_loader = dl_train, 
                                             val_loader = dl_val, 
                                             num_epochs = 1, 
                                             criterion = linfty,
                                             optimizer = optimizer)
torch.save(model.state_dict(), os.path.join(config['paths']['models'], f"GLSTMseq2seq_{past_step}_{future_step}.pt"))


  0%|                                                                           | 0/1 [00:00<?, ?it/s]

In [ ]:
def plot(model, 
         config,
         loss_training, 
         loss_validation, 
         name,
         dl_train = dl_train,
         dl_val = dl_val, 
         show = False):
    fig = px.line({"epochs": range(1,len(loss_training)+1), 
                                   "train": loss_training, 
                                   "validation": loss_validation}, 
                                  x = "epochs", 
                                  y = ["train", "validation"], 
                                  title= f"training loss for {name}")
    fig.add_vline(x = np.argsort(loss_validation)[0]+1)
    fig.add_hline(y = np.min(loss_validation))
    fig.write_html(os.path.join(config['paths']['fig'],'covid', f"loss_gnn_{name}.html"))
    if show:
        fig.show()
    model = model.cpu()
    model.device = torch.device('cpu')
    if config['dataset']['aggregate']:
        batch_train, y_train, adj_train = next(iter(dl_train))
        batch_val, y_val, adj_val = next(iter(dl_val))
        yh_train = model(batch_train.float().to(model.device), adj_train.to(model.device)).detach().numpy()
        yh_val = model(batch_val.float().to(model.device), adj_val.to(model.device)).detach().numpy()
    else:
        yh_train = []
        yh_val = []
        y_train = []
        y_val = []
        for key in dl_train.keys():
            batch_train, yt, adj_train = next(iter(dl_train[key]))
            batch_val, yv, adj_val = next(iter(dl_val[key]))
            yh_train.append(model(batch_train.float().to(model.device), adj_train.to(model.device)).cpu())
            yh_val.append(model(batch_val.float().to(model.device), adj_val.to(model.device)).cpu())
            y_train.append(yt)
            y_val.append(yv)
        
        yh_train = torch.cat(yh_train, -2).transpose(-1,-2)
        yh_val = torch.cat(yh_val, -2).transpose(-1,-2).detach().numpy()
        y_train = torch.cat(y_train, -1).detach().numpy()
        y_val = torch.cat(y_val, -1).detach().numpy()

    lenght = 3*y.shape[1]
    fig, ax = plt.subplots(nrows = y.shape[1], 
                           ncols = 2, 
                           constrained_layout = True,
                           figsize = (20,lenght))
    
    for day in range(y.shape[1]):
        ax[day, 0].plot(yh_train[0,day], label = "estimate")
        ax[day, 0].plot(y_train[0,day], label ="real")
    
        ax[day, 1].plot(yh_val[0,day], label = "estimate")
        ax[day, 1].plot(y_val[0,day], label ="real")
        ax[day, 0].legend()
        ax[day, 1].legend()
    
        ax[day, 0].title.set_text(f"day {day +1} train")
        ax[day, 1].title.set_text(f"day {day +1} validation")
    fig.suptitle(' Comparison between estimation and reality ', fontsize=20) 
    
    path = os.path.join(config['paths']['fig'],'covid', f"{name}.png")
    plt.savefig(path)
    if show:
        plt.show()
    plt.close(fig)


In [ ]:
plot(model,
     config,
     loss_training, 
     loss_validation, 
     f"GLSTMseq2seq_{past_step}_{future_step}", 
     show = True)